<img src="idal-logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">mIA3</h1></font>
<font color="#6E6E6E"><h2 align="left">Tarea Evaluable. Aprendizaje Profundo 1 y 2 (Parte 2 de 3).</h2></font> 

#### Elaborado por Felipe Ramírez Herrera

In [ ]:
# !pip install numpy
# !pip install panda

# !pip install torch
# !pip install torchtext
# !pip install torchmetrics
# !pip install spacy

# !pip install jupyter
# !pip install ipywidgets

# !pip install editdistance
# !pip install six
# !pip install typeguard

# !pip install matplotlib
# !pip install seaborn
# !pip install scikit-learn

# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm
# !python -m spacy download fr_core_news_sm
# !pip install wget

# !pip install --upgrade --force-reinstall torchtext

In [ ]:
import math
import random
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import numpy as np
import os
import pathlib
import csv
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torchtext.vocab import Vocab
import logging
import pandas as pd
import six
from typing import List, Tuple, Union
from argparse import Namespace
from tqdm.notebook import tqdm

import matplotlib.pyplot as mp
from matplotlib.colors import ListedColormap
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')

In [ ]:
# Formato de Mathplotlib

mp.rc('font', size=8)
mp.rc('axes', titlesize=8)
mp.rc('axes', labelsize=8)
mp.rc('xtick', labelsize=8)
mp.rc('ytick', labelsize=8)
mp.rc('legend', fontsize=8)
mp.rc('figure', titlesize=8)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:

torch.manual_seed(1234)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    !nvidia-smi

In [ ]:
def get_cuda_memory_available():
    if torch.cuda.is_available():
        t = torch.cuda.get_device_properties(DEVICE).total_memory
        r = torch.cuda.memory_reserved(DEVICE)
        a = torch.cuda.memory_allocated(DEVICE)
        f = (t - a - r) / t  # free inside reserved
        print("CUDA Available Memory: {0}".format(f))
    

In [ ]:
def count_params(model, return_int=False):
    params = sum([torch.prod(torch.tensor(x.shape)).item() for x in model.parameters() if x.requires_grad])
    if return_int:
        return params
    else:
        print("There are {:,} trainable parameters in this model.".format(params))

In [ ]:
# https://conferences.unite.un.org/UNCorpus/Home/DownloadOverview
# !cat /home/framirez/translation_multilingual/UNv1.0.6way.tar.gz.* | tar -xzf -

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import zipfile

# EN_corpus_file = "UNv1.0.6way_en.zip" # UN Parallel Corpus English (EN)
# ES_corpus_file = "UNv1.0.6way_es.zip" # UN Parallel Corpus Spanish (ES)
# FR_corpus_file = "UNv1.0.6way_fr.zip" # UN Parallel Corpus French (FR)

# corpus_source_folder = "/content/drive/MyDrive/"
# corpus_target_folder = "/tmp/"

# if (not os.path.exists(corpus_target_folder + EN_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + EN_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + ES_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + ES_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)

# if (not os.path.exists(corpus_target_folder + FR_corpus_file)):
#   with zipfile.ZipFile(corpus_source_folder + FR_corpus_file,"r") as zip_ref:
#       zip_ref.extractall(corpus_target_folder)


In [ ]:
# Use the cuDNN backend for 16-bit training and inference for CUDA-enabled GPUs
# torch.backends.cudnn.benchmark =  True
# torch.backends.cudnn.enabled =  True

In [ ]:
max_seq_length = 64 # Re-calculated further 
min_seq_length = 16 # Re-calculated further 

In [ ]:

NUMBER_OF_EPOCHS = 2
EARLY_STOPPING_EPOCHS = 3
CLIPPING_VALUE = 15                   # clipping value, or None to prevent gradient clipping
MAXIMUM_NUMBER_OF_SAMPLES = 500000
BATCH_SIZE = 32

# Lets GPU to get more cooler
INNER_GPU_REST_TIME = 0 # Rest time between epochs
OUTER_GPU_REST_TIME = 1 # Rest time between model training / validation processes


In [ ]:
import pandas as pd  

def generate_examples(src_file, tgt_a_file, tgt_b_file):
    data = []
    with open(src_file, encoding="utf-8") as src_f, open(tgt_a_file, encoding="utf-8") as tgt_a, open(tgt_b_file, encoding="utf-8") as tgt_b:
        for idx, (a, b, c) in enumerate(zip(src_f, tgt_a, tgt_b)):           
            if (a.isspace() | b.isspace() | c.isspace()):
                continue
            data.append({'text_en' : a, 'text_es' : b, 'text_fr' :c} )
    return pd.DataFrame.from_records(data=data)


In [ ]:
import re
from unicodedata import normalize

PAD_WORD = '<pad>'
UNK_WORD = '<unk>'  # Unknown token symbol
BOS_WORD = '<bos>'  # Begin-of-Sentence token symbol
EOS_WORD = '<eos>'  # End-of-Sentence token symbol

def clean_text(text : str):
    text = normalize('NFD', str(text).lower())
    text = re.sub('[^A-Za-z ]+', '', text)
    return text

pkl_dataset_file = "un_parallel_corpus.pkl"
un_ds = pd.DataFrame()

if os.path.exists(pkl_dataset_file):
    un_ds = pd.read_pickle(pkl_dataset_file) 
else:
    en_train_path = "./mnt/drive/UNv1.0.6way.en"
    es_train_path = "./mnt/drive/UNv1.0.6way.es"
    fr_train_path = "./mnt/drive/UNv1.0.6way.fr"
    df = generate_examples(en_train_path, es_train_path, fr_train_path)
    df['text_en'] = df['text_en'].apply(lambda row: clean_text(row))
    df['text_es'] = df['text_es'].apply(lambda row: clean_text(row))
    df['text_fr'] = df['text_fr'].apply(lambda row: clean_text(row))
    un_ds = df
    un_ds.to_pickle(pkl_dataset_file)
len(un_ds)


In [ ]:
un_ds.head(25)

In [ ]:
un_ds.head(-25)

In [ ]:
un_ds = un_ds.head(MAXIMUM_NUMBER_OF_SAMPLES)

In [ ]:
from enum import Enum

import torchtext

en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
es_tokenizer = get_tokenizer('spacy', language='es_core_news_sm')
fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')

Language = Enum('Language', ['EN', 'ES', 'FR'])

def yield_tokens(Lang: Language=Language.EN):
    for index, row in un_ds.iterrows():     
         if (Lang == Language.EN):
            yield en_tokenizer(str(row["text_en"]))  
         if (Lang == Language.ES):
            yield es_tokenizer(str(row["text_es"]))
         if (Lang == Language.FR):
           yield fr_tokenizer(str(row["text_fr"]))

       

In [ ]:
en_vocab_file = 'transformers_vocab_en.pth'
es_vocab_file = 'transformers_vocab_es.pth'
fr_vocab_file = 'transformers_vocab_fr.pth'

In [ ]:
import torchtext
import torchtext.data
import torchtext.data.datasets_utils
import torchtext.datasets

SPECIALS = [PAD_WORD, BOS_WORD, EOS_WORD,  UNK_WORD]

en_vocabulary = None
es_vocabulary = None
fr_vocabulary = None

if os.path.exists(en_vocab_file):
    en_vocabulary = torch.load(en_vocab_file)
else:
    en_vocabulary = build_vocab_from_iterator(yield_tokens(Language.EN), specials=SPECIALS)
    torch.save(en_vocabulary, en_vocab_file)

if os.path.exists(es_vocab_file):
    es_vocabulary = torch.load(es_vocab_file)
else:
    es_vocabulary = build_vocab_from_iterator(yield_tokens(Language.ES), specials=SPECIALS)
    torch.save(es_vocabulary, es_vocab_file)

if os.path.exists(fr_vocab_file):
    fr_vocabulary = torch.load(fr_vocab_file)
else:
    fr_vocabulary = build_vocab_from_iterator(yield_tokens(Language.FR), specials=SPECIALS)
    torch.save(fr_vocabulary, fr_vocab_file)

In [ ]:
print("EN Vocabulary Size = {0}".format(len(en_vocabulary)))
print("ES Vocabulary Size = {0}".format(len(es_vocabulary)))
print("FR Vocabulary Size = {0}".format(len(fr_vocabulary)))

In [ ]:
PAD_EN_IDX = en_vocabulary[PAD_WORD]
BOS_EN_IDX = en_vocabulary[BOS_WORD]
EOS_EN_IDX = en_vocabulary[EOS_WORD]
UNK_EN_IDX = en_vocabulary[UNK_WORD]


PAD_ES_IDX = es_vocabulary[PAD_WORD]
BOS_ES_IDX = es_vocabulary[BOS_WORD]
EOS_ES_IDX = es_vocabulary[EOS_WORD]
UNK_ES_IDX = es_vocabulary[UNK_WORD]

PAD_FR_IDX = fr_vocabulary[PAD_WORD]
BOS_FR_IDX = fr_vocabulary[BOS_WORD]
EOS_FR_IDX = fr_vocabulary[EOS_WORD]
UNK_FR_IDX = fr_vocabulary[UNK_WORD]

en_vocab_size = len(en_vocabulary) + 1
es_vocab_size = len(es_vocabulary) + 1
fr_vocab_size = len(fr_vocabulary) + 1

In [ ]:
def pad_or_truncate(tokenized_text, allocate : bool = True,  pad_index : int = PAD_EN_IDX, bos_index : int = BOS_EN_IDX, eos_index : int = EOS_EN_IDX):  
    result = []
    if len(tokenized_text) < max_seq_length:
        if (allocate):
            result = [bos_index] + tokenized_text + [eos_index]
            left = max_seq_length - len(result)
            padding = [pad_index] * left
            result = result + padding
        else:
            left = max_seq_length - len(tokenized_text)
            padding = [pad_index] * left
            result = tokenized_text + padding       
    else:
        raise Exception("pad_or_truncate: max_seq_length not computed properly")
    return result

In [ ]:
full_data_file = 'transformers_full.pth'
trn_data_file = 'transformers_trn.pth'
val_data_file = 'transformers_val.pth'
tst_data_file = 'transformers_tst.pth'

In [ ]:
len(un_ds)

In [ ]:
import random
import pickle
from collections import Counter 

MINIMUM_ALLOWED_SIZE_OF_SEQ = 5
MAXIMUM_ALLOWED_SIZE_OF_SEQ = 100

full_data = []
trn_subset, val_subset, tst_subset = [], [], []
random.seed(1234)
torch.manual_seed(1234)

max_len = 0
min_len = 4500

en_counter = Counter()
es_counter = Counter()
fr_counter = Counter()

en_lengths = []
es_lengths = []
fr_lengths = []

fe = os.path.exists(full_data_file)
te = os.path.exists(trn_data_file)
ve = os.path.exists(val_data_file)
tt = os.path.exists(tst_data_file)

if fe and te and ve and tt:
    with open(full_data_file, 'rb') as fp:
            full_data = pickle.load(fp)
    with open(trn_data_file, 'rb') as fp:
            trn_subset = pickle.load(fp)
    with open(val_data_file, 'rb') as fp:
            val_subset = pickle.load(fp)        
    with open(tst_data_file, 'rb') as fp:
            tst_subset = pickle.load(fp)           
    for (en_seq,es_seq,fr_seq) in full_data:     
        en_counter.update(en_seq)
        es_counter.update(es_seq)
        fr_counter.update(fr_seq)
        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 
        en_lengths.append(a)
        es_lengths.append(b)
        fr_lengths.append(c)
        max_len = max(max_len, a, b, c)    
        min_len = min(min_len, a, b, c)    
else:
    for idx, row in un_ds.iterrows():
        
        en_exp = row["text_en"].rstrip("\n")
        es_exp = row["text_es"].rstrip("\n")
        fr_exp = row["text_fr"].rstrip("\n")

        en_seq = [en_vocabulary[token] for token in en_tokenizer(en_exp)]
        es_seq = [es_vocabulary[token] for token in es_tokenizer(es_exp)]
        fr_seq = [fr_vocabulary[token] for token in fr_tokenizer(fr_exp)]

        a = len(en_seq)
        b = len(es_seq)
        c = len(fr_seq) 

        seq_min = min(a, b, c)
        seq_max = max(a, b, c)

        if seq_min  >= MINIMUM_ALLOWED_SIZE_OF_SEQ and seq_max <= MAXIMUM_ALLOWED_SIZE_OF_SEQ: 
            
            en_counter.update(en_seq)
            es_counter.update(es_seq)
            fr_counter.update(fr_seq)
       
            en_lengths.append(a)
            es_lengths.append(b)
            fr_lengths.append(c)

            max_len = max(max_len, seq_max)
            min_len = min(min_len, seq_min) 
            full_data.append((en_seq, es_seq, fr_seq))

    trn_subset, val_subset = train_test_split(full_data, test_size=0.3, train_size=0.7, random_state=1234, shuffle=True)
    val_subset, tst_subset = train_test_split(val_subset, test_size=0.33, train_size=0.67, random_state=1234, shuffle=True)

    with open(full_data_file, 'wb') as fp:
        pickle.dump(full_data, fp)
    with open(trn_data_file, 'wb') as fp:
        pickle.dump(trn_subset, fp)
    with open(val_data_file, 'wb') as fp:
        pickle.dump(val_subset, fp)
    with open(tst_data_file, 'wb') as fp:
        pickle.dump(tst_subset, fp)

if (max_seq_length < max_len):
    max_seq_length = max_len + 2

if (min_seq_length > min_len):
    min_seq_length = min_len + 2


print("Selected Records: {0}".format(len(full_data)))
print("MAX SEQ LEN {0}".format(max_seq_length))
print("MIN SEQ LEN {0}".format(min_seq_length))

size_of_trn_set_size = len(trn_subset)
size_of_val_set_size = len(val_subset)
size_of_tst_set_size = len(tst_subset)

print("Training tuples: {0} Validation tuples: {1} Testing tuples: {2}".format(size_of_trn_set_size, size_of_val_set_size, size_of_tst_set_size))


In [ ]:
fig = mp.figure(figsize=(8, 10))
ax0 = fig.add_subplot(311)
ax0.hist(en_lengths, rwidth=0.8, color='gray')
ax0.set_title("English Sentence Length")
ax0.set_xlabel("# Tokens in Sentence")

ax1 = fig.add_subplot(312)
ax1.hist(es_lengths, rwidth=0.8, color='gray')
ax1.set_title("Spanish Sentence Length")
ax1.set_xlabel("# Tokens in Sentence")

ax2 = fig.add_subplot(313)
ax2.hist(fr_lengths, rwidth=0.8, color='gray')
ax2.set_title("French Sentence Length")
ax2.set_xlabel("# Tokens in Sentence")

mp.tight_layout()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, es_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# Spanish Tokens")
mp.colorbar()
mp.show()

In [ ]:
mp.figure(figsize=(8,6))
mp.hist2d(en_lengths, fr_lengths, bins=max_seq_length-2, cmap='binary')
mp.title("Joint Distribution of Sentence Lengths")
mp.xlabel("# English Tokens")
mp.ylabel("# French Tokens")
mp.colorbar()
mp.show()

In [ ]:
def plot_top_words(counter : Counter, vocab: Vocab, k=20, ax=None):
    top_k = counter.most_common(k)
    tokens, freqs = zip(*reversed(top_k))
    
    words = [vocab.lookup_token(token) for token in tokens]

    if ax is None:
        mp.barh(words, freqs, color='gray')
    else:
        ax.barh(words, freqs, color='gray')

In [ ]:

fig = mp.figure(figsize=(14, 8))

ax0 = fig.add_subplot(131)
plot_top_words(en_counter, en_vocabulary, ax=ax0)
ax0.set_title("Top 20 English Words")
ax0.set_xlabel("Raw Frequency")

ax1 = fig.add_subplot(132)
plot_top_words(es_counter,es_vocabulary, ax=ax1)
ax1.set_title("Top 20 Spanish Words")
ax1.set_xlabel("Raw Frequency")

ax2 = fig.add_subplot(133)
plot_top_words(fr_counter,fr_vocabulary, ax=ax2)
ax2.set_title("Top 20 French Words")
ax2.set_xlabel("Raw Frequency")

mp.tight_layout()
mp.show()

In [ ]:
from  torch.nn.utils.rnn import pad_sequence

for en_id, es_id, fr_id in zip(en_vocabulary.lookup_indices(SPECIALS), es_vocabulary.lookup_indices(SPECIALS), fr_vocabulary.lookup_indices(SPECIALS)):
  assert en_id == es_id & es_id == fr_id


def tensor_transform(token_ids: List[int], bos_idx, eos_idx): # 57.1 segs
    list = [bos_idx] + token_ids + [eos_idx]                         
    return torch.as_tensor(list, device=DEVICE)

# def tensor_transform(token_ids: List[int], bos_idx, eos_idx):
#     return torch.cat((torch.tensor([bos_idx], device=DEVICE),
#                       torch.tensor(token_ids, device=DEVICE),
#                       torch.tensor([eos_idx], device=DEVICE)))

def generate_batch(data_batch):
    en_batch, es_batch, fr_batch = [], [], []
    for (en_item, es_item, fr_item) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)     
        en_batch.append(en_t)
        es_batch.append(es_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_FR_IDX, batch_first=False)
    return en_batch, es_batch, fr_batch

def generate_batch_EN_ES(data_batch):
    en_batch, es_batch = [], []
    for (en_item, es_item, _) in data_batch:     
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        es_t = tensor_transform(es_item, BOS_ES_IDX, EOS_ES_IDX)
        en_batch.append(en_t)
        es_batch.append(es_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    es_batch = pad_sequence(es_batch, padding_value=PAD_ES_IDX, batch_first=False)
    return en_batch, es_batch

def generate_batch_EN_FR(data_batch):
    en_batch, fr_batch = [], []
    for (en_item, _, fr_item) in data_batch:        
        en_t = tensor_transform(en_item, BOS_EN_IDX, EOS_EN_IDX)
        fr_t = tensor_transform(fr_item, BOS_FR_IDX, EOS_FR_IDX)
        en_batch.append(en_t)
        fr_batch.append(fr_t)
    en_batch = pad_sequence(en_batch, padding_value=PAD_EN_IDX, batch_first=False)
    fr_batch = pad_sequence(fr_batch, padding_value=PAD_ES_IDX, batch_first=False)
    return en_batch, fr_batch

In [ ]:
from torch.utils.data import Dataset

class LanguageDataset(Dataset):
    
    def __init__(self, subset, length):
        self.length = length
        self.subset = subset

    def __getitem__(self, idx):
        return self.subset[idx]

    def __len__(self):
        return self.length

In [ ]:

Bilingual_EN_to_ES_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_ES)
Bilingual_EN_to_ES_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES)
Bilingual_EN_to_ES_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_ES)

Bilingual_EN_to_FR_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch_EN_FR)
Bilingual_EN_to_FR_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR)
Bilingual_EN_to_FR_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch_EN_FR)

Trilingual_trnset = DataLoader(LanguageDataset(trn_subset, size_of_trn_set_size), batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
Trilingual_valset = DataLoader(LanguageDataset(val_subset, size_of_val_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)
Trilingual_tstset = DataLoader(LanguageDataset(tst_subset, size_of_tst_set_size), batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)

batches_for_training = math.ceil( size_of_trn_set_size / BATCH_SIZE)
batches_for_validation = math.ceil( size_of_val_set_size / BATCH_SIZE)


In [ ]:
import sys

size_to_MB = 1024 * 1024

def getSizeOf(a : torch.Tensor):
    return sys.getsizeof(a) + torch.numel(a) * a.element_size()

def ElementsOf(a : torch.Tensor):
    return torch.numel(a)
    
torch.cuda.empty_cache()
total = 0

for (src, tgt_ES, tgt_FR) in iter(Trilingual_trnset):

    total += getSizeOf(src) / size_to_MB
    total += getSizeOf(tgt_ES) / size_to_MB
    total += getSizeOf(tgt_FR) / size_to_MB

    del src
    del tgt_ES
    del tgt_FR

print("Tamaño estimado del conjunto de datos: {} Mbytes".format(total))

torch.cuda.empty_cache()

## Funcionalidad compartida

In [ ]:
import torch

class GarbageCollector:

    def __init__(self):
        super().__init__()
        self.collected = False
        self.registry = []

    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.collected = True

        for T in self.registry:
            del T

        torch.cuda.empty_cache()

    def Register(self, T : torch.Tensor):
        if (not self.collected):
            self.registry.append(T)

In [ ]:
# Fragmento de código. Funciones que despliegan las curvas de desempeño para el modelo
# Se reutilizan a lo largo del ejercicio.

import matplotlib.pyplot as plt


PLOT_X = 10
PLOT_Y = 2.7    

def plot_accuracy_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_acc'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_pplx_curve(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Perplejidad del modelo {0}'.format(name))
    ax.plot(history['train_ppl'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_ppl'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Perplejidad')
    ax.legend()
    plt.show()


def plot_accuracy_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_joint_accm'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_joint_accm'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()


def plot_loss_curve_dual_transformer(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_joint_loss'], label='Entrenamiento', linestyle='solid', lw=2.5)
    ax.plot(history['valid_joint_loss'], label='Validación', linestyle='dashed', lw=2.5)
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

def plot_accuracy_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Exactitud del modelo {0}'.format(name))
    ax.plot(history['train_acc_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_acc_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_acc_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_acc_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_yticks(np.arange(0, 1, 1 / 10))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Exactitud')
    ax.legend()
    plt.show()

def plot_loss_curve_dual_transformer_both(name, history):
    fig, ax = plt.subplots(figsize=(PLOT_X, PLOT_Y), layout='constrained')
    ax.set_title('Pérdida del modelo {0}'.format(name))
    ax.plot(history['train_loss_a'], label='Entrenamiento (EN-ES)')
    ax.plot(history['train_loss_b'], label='Entrenamiento (EN-FR)')
    ax.plot(history['valid_loss_a'], label='Validación (EN-ES)')
    ax.plot(history['valid_loss_b'], label='Validación (EN-FR)')
    ax.set_xticks(np.arange(1, NUMBER_OF_EPOCHS + 1, 1))
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Pérdida')
    ax.legend()
    plt.show()

In [ ]:
import time

def let_gpu_rest(minutes):
    if minutes > 0:
        time.sleep(minutes * 60)    

In [ ]:
def measure_data_sizing():
    size_of_epoch = 0
    batches = 0
    elements = 0
    for (src, tgt_ES, tgt_FR) in iter(Trilingual_trnset):
        tgt_a_input = tgt_ES[:-1, :]
        tgt_b_input = tgt_FR[:-1, :]
        src_mask, tgt_a_mask, tgt_b_mask, src_padding_mask, tgt_a_padding_mask, tgt_b_padding_mask = create_mask_for_dual_decoder(src, tgt_a_input, tgt_b_input)
        elements += ElementsOf(src)
        elements += ElementsOf(tgt_ES)
        elements += ElementsOf(tgt_FR)
        size_of_epoch += getSizeOf(src)
        size_of_epoch += getSizeOf(tgt_ES)
        size_of_epoch += getSizeOf(tgt_FR)
        size_of_epoch += getSizeOf(tgt_a_input)
        size_of_epoch += getSizeOf(tgt_b_input)
        size_of_epoch += getSizeOf(src_mask)
        size_of_epoch += getSizeOf(tgt_a_mask)
        size_of_epoch += getSizeOf(tgt_b_mask)
        size_of_epoch += getSizeOf(src_padding_mask)
        size_of_epoch += getSizeOf(tgt_a_padding_mask)
        size_of_epoch += getSizeOf(tgt_b_padding_mask)
        del src
        del tgt_ES
        del tgt_FR
        del tgt_a_input
        del tgt_b_input
        del src_mask
        del tgt_a_mask
        del tgt_b_mask
        del src_padding_mask
        del tgt_a_padding_mask
        del tgt_b_padding_mask
        batches += 1
        
    print("Tamaño estimado del conjunto de datos: {0} Mbytes en {1} batches = {2} elements".format(size_of_epoch / size_to_MB, batches, elements))
    torch.cuda.empty_cache()    


### Funcionalidad compartida para los modelos

In [ ]:
def init_transformer_model(model : nn.Module):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

In [ ]:
def masked_accuracy(label, pred, tgt_pad_idx = PAD_ES_IDX):
  pred = torch.argmax(pred, dim=-1)
  match = label.eq(pred)
  mask = label.ne(tgt_pad_idx)
  match = match & mask
  match = match.type(torch.float32) 
  mask =  mask.type(torch.float32)
  return torch.sum(match)/torch.sum(mask)

In [ ]:



# Basado en https://github.com/pytorch/pytorch/blob/main/torch/nn/modules/transformer.py

### Componentes comunes

#### Embedding

NameError: name 'nn' is not defined

#### Encoder

#### Decoder

#### Transformer (single decoder)

#### Transformer (dual decoder)

### Seq2Seq Transformer

#### Training and evaluation routines

#### Full Epoch Process

### Seq2Seq Dual Decoder Transfomer

#### Training and evaluation routines (dual decoder)

#### Full Epoch Process (dual decoder)

### Experimentos

In [ ]:

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

base_EN_ES_model_path = 'transformer_base_en_es_{0}.pt'
base_EN_FR_model_path = 'transformer_base_en_fr_{0}.pt'
# custom_EN_ES_model_path = 'transformer_cstm_en_es_{0}.pt'
# custom_EN_FR_model_path = 'transformer_cstm_en_fr_{0}.pt'
dual_model_path = 'transformer_cstm_en_es_fr_{0}.pt'
dual_model_path_WS = 'transformer_cstm_en_es_fr_weight_sharing_{0}.pt'

transformer_base_EN_to_ES = Seq2SeqTransformer("Bilingual_EN_ES (Lib)", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, en_vocab_size, es_vocab_size, FFN_HID_DIM)
transformer_base_EN_to_FR = Seq2SeqTransformer("Bilingual_EN_FR (Lib)",NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, en_vocab_size, fr_vocab_size, FFN_HID_DIM)
transformer_dual_EN_to_ES_and_FS = DoubleTaskSeq2SeqTransformer("Trilingual_EN_ES_FR", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, en_vocab_size, es_vocab_size, fr_vocab_size, FFN_HID_DIM)


transformer_dual_EN_to_ES_and_FS_with_WS = DoubleTaskSeq2SeqTransformer("Trilingual_EN_ES_FR_WS", NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, en_vocab_size, es_vocab_size, fr_vocab_size, FFN_HID_DIM, weight_sharing=True)





In [ ]:
random.seed(1234)
torch.manual_seed(1234)
transformer_base_EN_to_ES.apply(init_transformer_model)

In [ ]:
count_params(transformer_base_EN_to_ES)

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
transformer_base_EN_to_FR.apply(init_transformer_model)

In [ ]:
count_params(transformer_base_EN_to_FR)

In [ ]:
random.seed(1234)
torch.manual_seed(1234)
transformer_dual_EN_to_ES_and_FS.apply(init_transformer_model)

In [ ]:
count_params(transformer_dual_EN_to_ES_and_FS)

In [ ]:

random.seed(1234)
torch.manual_seed(1234)
transformer_dual_EN_to_ES_and_FS_with_WS.apply(init_transformer_model)

In [ ]:
count_params(transformer_dual_EN_to_ES_and_FS_with_WS)

In [ ]:
loss_fn_base_EN_to_ES = torch.nn.CrossEntropyLoss(ignore_index=PAD_ES_IDX)
loss_fn_base_EN_to_FR = torch.nn.CrossEntropyLoss(ignore_index=PAD_ES_IDX)



loss_fn_dual_decoder_EN_to_ES = torch.nn.CrossEntropyLoss(ignore_index=PAD_ES_IDX)
loss_fn_dual_decoder_EN_to_FR = torch.nn.CrossEntropyLoss(ignore_index=PAD_FR_IDX)


loss_fn_dual_decoder_EN_to_ES_WS = torch.nn.CrossEntropyLoss(ignore_index=PAD_ES_IDX)
loss_fn_dual_decoder_EN_to_FR_WS = torch.nn.CrossEntropyLoss(ignore_index=PAD_FR_IDX)

optimizer_base_EN_to_ES = torch.optim.Adam(transformer_base_EN_to_ES.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
optimizer_base_EN_to_FR = torch.optim.Adam(transformer_base_EN_to_FR.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

scheduler_EN_ES = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_base_EN_to_ES, 'min')
scheduler_EN_FR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_base_EN_to_FR, 'min')

optimizer_custom_EN_to_ES_and_FR = torch.optim.Adam(transformer_dual_EN_to_ES_and_FS.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
optimizer_custom_EN_to_ES_and_FR_WS = torch.optim.Adam(transformer_dual_EN_to_ES_and_FS_with_WS.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

scheduler_EN_to_ES_and_FR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_custom_EN_to_ES_and_FR, 'min')
scheduler_EN_to_ES_and_FR_WS = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_custom_EN_to_ES_and_FR_WS, 'min')

In [ ]:
    
# measure_data_sizing()

## Ejecución de los modelos basados en un único decodificador

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, pad_symbol):

    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)

    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = generate_square_subsequent_mask(ys.size(0)).type(torch.bool).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == pad_symbol:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: nn.Module, src_sentence: str, src_lang : Language, tgt_lang : Language):
    model.eval()

    start_symbol = 0
    pad_symbol = 0

    src_vocab = None
    tgt_vocab = None
    src_tokenizer = None

    match (src_lang):
        case Language.EN:
            src_vocab = en_vocabulary
            src_tokenizer = en_tokenizer
        case Language.ES:
            src_vocab = es_vocabulary
            src_tokenizer = es_tokenizer
        case Language.FR:
            src_vocab = fr_vocabulary
            src_tokenizer = fr_tokenizer
        case _:
            raise RuntimeError("invalid src language")

    match (tgt_lang):
        case Language.EN:
            tgt_vocab = en_vocabulary
            start_symbol = PAD_EN_IDX
            pad_symbol = BOS_EN_IDX
        case Language.ES:
            tgt_vocab = es_vocabulary
            start_symbol = PAD_ES_IDX
            pad_symbol = BOS_ES_IDX
        case Language.FR:
            tgt_vocab = fr_vocabulary
            start_symbol = PAD_FR_IDX
            pad_symbol = BOS_FR_IDX
        case _:
            raise RuntimeError("invalid src language")

    src_seq = torch.Tensor([src_vocab[token] for token in src_tokenizer(clean_text(src_sentence))])
    src_seq = src_seq.view(-1, 1)
    num_tokens = src_seq.shape[0]
    src_seq_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src_seq, src_seq_mask, max_len=num_tokens + 5, start_symbol=start_symbol, pad_symbol=pad_symbol).flatten()

    return " ".join(tgt_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace(BOS_WORD, "").replace(EOS_WORD, "")


def translate_bleu(model: nn.Module, sequence):
    model.eval()
    start_symbol = 0
    pad_symbol = 0
    src_seq = torch.Tensor(sequence)
    src_seq = src_seq.view(-1, 1)
    num_tokens = src_seq.shape[0]
    src_seq_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src_seq, src_seq_mask, max_len=num_tokens + 5, start_symbol=start_symbol, pad_symbol=pad_symbol).flatten()
    return tgt_tokens

In [ ]:
from torchtext.data.metrics import bleu_score

max_printing_results = 5

tst_models = [transformer_base_EN_to_ES, transformer_base_EN_to_FR]
tst_slangs = [Language.EN, Language.EN] 
tst_tlangs = [Language.ES, Language.FR] 


for m, s_lang, t_lang in zip(tst_models, tst_slangs, tst_tlangs):
    print ("Model: {0}".format(m.getName()))
    exact = []
    candidate = []
    i = 0
    for (src, tgt_ES, tgt_FR) in iter(tst_subset):

        src_EN_tokens = en_vocabulary.lookup_tokens(src)
        tgt_ES_tokens = es_vocabulary.lookup_tokens(tgt_ES)
        tgt_FR_tokens = fr_vocabulary.lookup_tokens(tgt_FR)

        src_txt = " ".join(src_EN_tokens).replace(BOS_WORD, "").replace(EOS_WORD, "")
        tgt_es_txt = " ".join(tgt_ES_tokens).replace(BOS_WORD, "").replace(EOS_WORD, "")
        tgt_fr_txt = " ".join(tgt_FR_tokens).replace(BOS_WORD, "").replace(EOS_WORD, "")
       
        x = translate_bleu(m, src)

        pred_txt = None
        ground_truth = None
        output_tokens = None
        exact_tokens = None
        if t_lang == Language.ES:
            exact_tokens = tgt_ES_tokens
            ground_truth = tgt_es_txt
            output_tokens = es_vocabulary.lookup_tokens(list(x.cpu().numpy()))
            pred_txt = " ".join(output_tokens).replace(BOS_WORD, "").replace(EOS_WORD, "").replace(PAD_WORD, " ")

        if t_lang == Language.FR:
            exact_tokens = tgt_FR_tokens
            ground_truth = tgt_fr_txt
            output_tokens = fr_vocabulary.lookup_tokens(list(x.cpu().numpy()))
            pred_txt = " ".join(output_tokens).replace(BOS_WORD, "").replace(EOS_WORD, "").replace(PAD_WORD, " ")

        if len(exact_tokens) > 0 and len(output_tokens) > 0:
            exact.append(exact_tokens)
            candidate.append(output_tokens)

        if (i < max_printing_results):
            print("{0}.SRC_SEQ: {1}".format(i+1, src_txt))
            print("{0}.TGT_SEQ: {1}".format(i+1, ground_truth))
            print("{0}.OUT_SEQ: {1}".format(i+1, pred_txt))
        i+=1

        if (i > 1):
            break

            
    print(candidate)
    print(exact)
    print("BLEU Score for {0} = {1}".format(m.getName(), bleu_score(candidate, exact)))


In [ ]:



display(result)

torch.cuda.empty_cache()


